<a href="https://colab.research.google.com/github/Winest-Nigeria/fgan-I-298-R1/blob/main/2023/Triplet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2 as pdf
import string

In [ ]:
# Path to your PDF file
pdf_file_path = '/content/sample_data/test_usecase_document.pdf'

In [ ]:
# Open the PDF file
with open(pdf_file_path, 'rb') as pdf_file:
  # Create a PDF reader object
  pdf_reader = pdf.PdfReader(pdf_file)

  # Initialize an empty string to store the extracted text
  pdf_text = ''

  # Iterate through each page and extract text
  for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      pdf_text += page.extract_text()


In [ ]:
pdf_text[:100]

' \n  \n7.1  Import and export of knowledge in an autonomous network  \n  \nUse case ID  AN-usecase -001 '

In [ ]:
# Remove all puntuation in text
text= ''.join([c for c in pdf_text if c not in string.punctuation])

In [ ]:
text[:100]

' \n  \n71  Import and export of knowledge in an autonomous network  \n  \nUse case ID  ANusecase 001  \nU'

In [ ]:
len(text)

9655

In [ ]:

#Tokenize the pdf text
tokens = text.split()

In [ ]:

#show first 10 elements of token list
tokens[:10]

['71',
 'Import',
 'and',
 'export',
 'of',
 'knowledge',
 'in',
 'an',
 'autonomous',
 'network']

In [ ]:
#number of elements in tokens list
len(tokens)

1474

In [ ]:
#import nltk library and download necessary files
import nltk
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
#Import nltk library stopwords
from nltk.corpus import stopwords

In [ ]:
#Remove stopwords from tokens
tokens=[word for word in tokens if word.lower() not in stopwords.words('english')]

In [ ]:
#Number of elements in new tokens list
len(tokens)

945

In [ ]:
#remove numbers and single character from tokens list
tokens = [word for word in tokens if not (word.isdigit() or len(word) == 1)]

In [ ]:
tokens = [word for word in tokens if not (word == 'Use' or word == 'use' or word == 'case')]

In [ ]:
len(tokens)

793

In [ ]:
tokens[:10]

['Import',
 'export',
 'knowledge',
 'autonomous',
 'network',
 'ID',
 'ANusecase',
 'description',
 'satisfy',
 'key']

In [ ]:

#import spacy library
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
word_list = tokens
# Join the words into a text string
text = ' '.join(word_list)

# Process the text with spaCy
doc = nlp(text)

# Extract nouns
noun = [token.text for token in doc if token.pos_ == "NOUN"]

# Print the extracted nouns
print(noun[:100])

['Import', 'export', 'knowledge', 'network', 'description', 'concepts', 'ANs', 'evolution', 'experimentation', 'adaptation', 'intervention', 'knowledge', 'knowledge', 'representation', 'data', 'environment', 'system', 'actions', 'configuration', 'options', 'measurement', 'parameters', 'elements', 'logic', 'scenarios', 'knowledge', 'actors', 'scenarios', 'steps', 'peer', 'entities', 'components', 'eg', 'evolution', 'exploration', 'configuration', 'automation', 'loops', 'consumption', 'components', 'components', 'peer', 'entities', 'category', 'scenario', 'behaviour', 'requirements', 'requirements', 'exchange', 'knowledge', 'components', 'ANUC01', 'optimization', 'knowledge', 'bases', 'optimizations', 'knowledge', 'bases', 'access', 'policies', 'arity', 'storage', 'interconnection', 'knowledge', 'bases', 'relation', 'problems', 'solutions', 'knowledge', 'ANUC01', 'creation', 'knowledge', 'bases', 'ion', 'humans', 'machines', 'contents', 'statistics', 'knowledge', 'components', 'network',

In [ ]:
len(noun)

398

In [ ]:

# using nltk to retrieve the part of speech
parts_of_speech = pos_tag(noun)
len(parts_of_speech)

398

In [ ]:
parts_of_speech[:10]

[('Import', 'NNP'),
 ('export', 'NN'),
 ('knowledge', 'NN'),
 ('network', 'NN'),
 ('description', 'NN'),
 ('concepts', 'NNS'),
 ('ANs', 'NNP'),
 ('evolution', 'NN'),
 ('experimentation', 'NN'),
 ('adaptation', 'NN')]

In [ ]:
#extract nouns from the tokens list
nouns = list(set(filter(lambda x: x[1]=="NN" , parts_of_speech)))

In [ ]:
nouns

In [ ]:
len(nouns)

74

**For Colab users:**
If you are using colab, add your credentials here and uncomment these

In [ ]:
uri = "neo4j+s://59fc3694.databases.neo4j.io:7687"
user = "neo4j"
password = "Yl2OjSlWtbYUfPUJdNgevZWGBHzjtP234TuWD7rePsk"

In [ ]:
%pip install neo4j

In [ ]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

**Define helper class**

In [ ]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    #Vishnu: 1 Aug 2022: uses labels instead of properties
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #        (Note that rel still uses properties and not labels)
    def create_node_with_usecase_label(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, actor_name)
            if not node_already_exists:
                #node itself doesnt exist (not to mention the use case label), Lets add.
                result = session.write_transaction(
                    self._create_and_return_node_label, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'],
                                                                 usecase=row['n1usecase']))
                    if not (usecase_id in row['n1usecase']):
                      print("adding label "
                                +usecase_id+
                                " in addition to {usecase}".format(usecase=row['n1usecase']))
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node_label, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break

    #Vishnu: 1 Aug 2022: created this f() to query nodes based on a name
    #                    and return with usecase labels
    @staticmethod
    def _find_and_return_existing_node_label(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, labels(n1) as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to create nodes with usecase label
    @staticmethod
    def _create_and_return_node_label(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 :" + usecase_id + " { name: '" + actor_name + "'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: created this f() to add use case labels to existing nodes
    #                    Note that the existing labels are left there.
    @staticmethod
    def _add_usecase_and_return_existing_node_label(tx, actor_name, usecase_id):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1 :" + usecase_id +" "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to return all use case actors based on labels
    #                    as against properties
    @staticmethod
    def _find_and_return_all_usecase_actors_label(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE '" +usecase_id+ "' in labels(n) "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 1 Aug 2022: created this f() to list all nodes with usecase label
    #                    instead of properties
    def find_all_usecase_actors_label(self, usecase_id):
        all_actors = []
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors_label, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))
                print(row)
                all_actors.append(row)
        all_actors.sort()
        return all_actors

    #Vishnu: 1 Aug 2022: Created to use label for use case.
    #CAUTION: Assumes that actors already exists with label as use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase_label(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case,
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel_label, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'],
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))

    #Vishnu: 1 Aug 2022
    #assumption: nodes already exist
    #modified from _create_and_return_rel to use labels for n.usecase
    @staticmethod
    def _create_and_return_rel_label(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND '"+ usecaseid+"' in labels(n1) "
            " AND '"+ usecaseid+"' in labels(n2) "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"],
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for n.usecases.
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #Vishnu: 22 Jun 2022: refactored, uses properties instead of labels
    #Vishnu: 1 Jun 2022: created this f() to create nodes with usecase label
    #Vishnu: 2 Jun 2022: added- check if it already exists.
    #Vishnu: 2 Jun 2022: added- check the usecase duplication, adds a usecase if it doesnt exist.
    #otherwise ignores the command.
    def create_node_with_usecase(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node, actor_name)
            if not node_already_exists:
                result = session.write_transaction(
                    self._create_and_return_node, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'],
                                                                 usecase=row['n1usecase']))
                    s=row['n1usecase'].split(',')
                    if not (usecase_id in s):
                      print("adding prop "+usecase_id+ " in addition to "+row['n1usecase'])
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    #Vishnu: 2 Jun 2022: created this f() to query nodes and return with usecase prop
    @staticmethod
    def _find_and_return_existing_node(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, n1.usecase as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _create_and_return_node(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 "+"{ name: '"+actor_name+"', usecase: '"+usecase_id+"'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    #Vishnu: 1 Jun 2022: created this f() to add use case prop to existing nodes
    #CAUTION: does not check for duplication, use create_node_with_prop instead.
    def add_usecase_existing_node(self, actor_name, usecase_id):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._add_usecase_and_return_existing_node, actor_name, usecase_id)

            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added usecase to node: {n1}".format(n1=row['n1']))

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _add_usecase_and_return_existing_node(tx, actor_name, usecase_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1.usecase = n1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    #CAUTION: Assumes that actors already exists with property as n.use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case,
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'],
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))

    #Vishnu: 1 Aug 2022: at this moment this f() uses labels instead of properties for n.usecases.
    #                    Note that rels still uses properties.
    #        1 Aug 2022: TBD: create a property-version (as against label) of the same thing.
    #Vishnu: 21 Jun 2022: created this f() to create relationships with usecase label
    #if this works, this is the only function that needed to be called.
    def create_actors_relationship_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
            #process actor1
            self.create_node_with_usecase_label(actor1_name, usecase_id)
            #process actor2
            self.create_node_with_usecase_label(actor2_name, usecase_id)
            #process relation
            self.create_rel_with_usecase_label(actor1_name, rel_name, actor2_name, usecase_id)

    #Vishnu: 23 June 2022: created
    #looks for a rel under a use case between 2 given nodes.
    #normally, we expect only 0 or 1 unique entry.
    @staticmethod
    def _find_and_return_this_existing_rels(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "and   (r1.name ='" + rel_name + "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        return [row["r1name"] for row in result]


    #Vishnu: 21 June 2022: created this supporting f()
    #return for all relationship between given two nodes.
    @staticmethod
    def _find_and_return_all_existing_rels_for_uc(tx, actor1_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        try:
            return [{
                        "n1name": row["n1name"],
                        "n2name": row["n2name"],
                        "r1name": row["r1name"],
                        "r1usecase": row["r1usecase"],
                    }
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 21 June 2022: created this supporting f()
    #adds a name for an existing relationship between given two nodes.
    @staticmethod
    def _add_usecase_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "')"
            "set    r1.usecase = r1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
        )
        result = tx.run(query)


    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists,
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_node(tx, actor_name, propertyName, propertyVal):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1."+propertyName+" = '"+propertyVal +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists,
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name,
                                                propertyName, propertyVal):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.name ='" + rel_name + "') "
            "set   r1."+propertyName+" = '"+propertyVal +"' "
            "return r1"
        )
        result = tx.run(query)
        try:
            return [{"r1": row["r1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created
    #assumption: nodes already exist
    @staticmethod
    def _create_and_return_rel(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND n1.usecase CONTAINS '"+ usecaseid+"' "
            " AND n2.usecase CONTAINS '"+ usecaseid+"' "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"],
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Jun 2022: created this f() to list all nodes with usecase label
    def find_all_usecase_actors(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    @staticmethod
    def _find_and_return_all_usecase_actors(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE (n.usecase = '"+usecase_id+"') "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 21 Jun 2022: created this f() to cleanup the DB
    #CAUTION: will delete everything
    def cleanup_db(self):
        with self.driver.session() as session:
            result = session.read_transaction(
                self._count_nodes_in_db)
            for row in result:
                print("Found {row} actors, deleting! ".format(row=row['count']))
            result = session.write_transaction(
                self._cleanup_db)

    @staticmethod
    def _count_nodes_in_db(tx):
        query = (
            "MATCH (n) "
            "return count(n) as count"
        )
        result = tx.run(query)
        try:
            return [{"count": row["count"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created this f()
    #(over)writes a prop to a node
    def writePropToNode(self, actor_name, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_node,
                    actor_name,
                    propertyName,
                    propertyVal)

            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to node: {n1}".format(n1=row['n1']))

    #Vishnu: 22 June 2022: created this f()
    #(over)writes a prop to a rel
    def writePropToRel(self, a1, rel_name, a2, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_rel,
                    a1, rel_name, a2,
                    propertyName,
                    propertyVal)

            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to rel: {r1}".format(r1=row['r1']))

    @staticmethod
    def _cleanup_db(tx):
        query = (
            "MATCH (n) "
            "detach delete n"
        )
        result = tx.run(query)
        try:
            return
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 29 Aug 2022
    #https://github.com/vrra/FGAN-Build-a-thon-2022/issues/52
    def del_node_in_db(self, node_name):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, node_name)
            if not node_already_exists:
                print(f"\"{node_name}\" does not exist! ")
            result = session.write_transaction(
                self._del_node_in_db, node_name)
            for row in result:
                    print("Deleted node: {n1}".format(n1=row['name']))

    #Vishnu: 29 Aug 2022
    #https://github.com/vrra/FGAN-Build-a-thon-2022/issues/52
    @staticmethod
    def _del_node_in_db(tx, actor1_name):
        query = (
            "MATCH (n) "
            "WHERE (n.name = '"+actor1_name+"') "
            "with n, n.name as name "
            "detach delete n "
            "return name"
        )
        result = tx.run(query)
        try:
            return [{"name": row["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    # Created this f() to return each actors relationship name
    @staticmethod
    def _find_and_return_all_usecase_actors_relationship_name(tx, actor):
        query = (
            "MATCH ({name: '"+actor+"'})-[r]->(o) RETURN r.name, o.name"
        )
        result = tx.run(query)
        return [row.values() for row in result]

    # Created this f() to list all actors relationship name
    def find_actor_relationship_name(self, actor):
        with self.driver.session() as session:
            return session.read_transaction(self._find_and_return_all_usecase_actors_relationship_name, actor)

In [ ]:
app = App(uri, user, password)

In [ ]:
for word in nouns:
  app.create_node_with_usecase_label(word[0], 'usecase001')